In [13]:
import numpy as np
import matplotlib.pyplot as plt
import math

earthRadiusInMeters = 6371000
earthR = 6371

In [14]:
# Coordinates of  A:
LatA = 48.8674546593467
LonA = 2.355035664087364

In [15]:
# Coordinates of  B: 
LatB = 48.86649526947721
LonB = 2.355244287413478

In [16]:
# Coordinates of  C
LatC = 48.8670763004335
LonC = 2.356347186129253

In [17]:
# Theoretical coordinates of the Blind Node
LatD_th = 48.86691223210363
LonD_th = 2.355337952126204

In [18]:
# Distances measured Blind node to anchor nodes [km]
DistA = 64.2402 / 1000
DistB = 46.8676 / 1000
DistC = 76.0414 / 1000

In [19]:
DistA

0.0642402

In [20]:
DistB

0.0468676

In [21]:
DistC

0.0760414

In [24]:
def intersectCircles(LatA, LonA, DistA, LatB, LonB, DistB, LatC, LonC, DistC):
        """
           Function borrowed from https://gis.stackexchange.com/questions/66/trilateration-using-3-latitude-longitude-points-and-3-distances
           intersectCircles: returns the coordinates of a point over the distance to 3 reference points
           Provide the coordinates [decimal degrees] of the reference points,
           and distances [km] to reference points,
           to obtain the coordinates on the terrestrial globe modeled by an authalic sphere.
          
           Example:
           long format
           [lat, lon] = intersectCircles (37.418436, -121.963477, 0.265710701754, 37.417243, -121.961889, 0.234592423446, 37.418692, -121.960194, 0.0548954278262)
           lat =
           37.419102373825389
           lon =
           -1.219605792083924e + 02
        """
        # assuming elevation = 0
        earthR = 6371
        
        # Convert geodetic Lat/Long to ECEF xyz
        #   1. Convert Lat/Long to radians
        #   2. Convert Lat/Long(radians) to ECEF
        xA = earthR *(math.cos(math.radians(LatA)) * math.cos(math.radians(LonA)))
        yA = earthR *(math.cos(math.radians(LatA)) * math.sin(math.radians(LonA)))
        zA = earthR *(math.sin(math.radians(LatA)))

        xB = earthR *(math.cos(math.radians(LatB)) * math.cos(math.radians(LonB)))
        yB = earthR *(math.cos(math.radians(LatB)) * math.sin(math.radians(LonB)))
        zB = earthR *(math.sin(math.radians(LatB)))

        xC = earthR *(math.cos(math.radians(LatC)) * math.cos(math.radians(LonC)))
        yC = earthR *(math.cos(math.radians(LatC)) * math.sin(math.radians(LonC)))
        zC = earthR *(math.sin(math.radians(LatC)))

        P1 = np.array([xA, yA, zA])
        P2 = np.array([xB, yB, zB])
        P3 = np.array([xC, yC, zC])

        #from wikipedia
        #transform to get circle 1 at origin
        #transform to get circle 2 on x axis
        ex = (P2 - P1)/(np.linalg.norm(P2 - P1))
        i = np.dot(ex, P3 - P1)
        ey = (P3 - P1 - i*ex)/(np.linalg.norm(P3 - P1 - i*ex))
        ez = np.cross(ex,ey)
        d = np.linalg.norm(P2 - P1)
        j = np.dot(ey, P3 - P1)

        #from wikipedia
        #plug and chug using above values
        x = (pow(DistA,2) - pow(DistB,2) + pow(d,2))/(2*d)
        y = ((pow(DistA,2) - pow(DistC,2) + pow(i,2) + pow(j,2))/(2*j)) - ((i/j)*x)

        # only one case shown here
        z = np.sqrt(pow(DistA,2) - pow(x,2) - pow(y,2))

        #triPt is an array with ECEF x,y,z of trilateration point
        triPt = P1 + x*ex + y*ey + z*ez

        #convert back to lat/long from ECEF
        #convert to degrees
        lat = math.degrees(math.asin(triPt[2] / earthR))
        lon = math.degrees(math.atan2(triPt[1],triPt[0]))

        # print(lat, lon)
        
        return lat, lon

In [25]:
# Test Case 
[lat, lon] = intersectCircles (37.418436, -121.963477, 0.265710701754, 37.417243, -121.961889, 0.234592423446, 37.418692, -121.960194, 0.0548954278262)

37.41910237382539 -121.96057920839233


In [26]:
[lat, lon] = intersectCircles(LatA, LonA, DistA, LatB, LonB, DistB, LatC, LonC, DistC)

48.86691249963017 2.3553379517040205


In [23]:
# Trilateration function borrowed from 
# Stackoverflow "https://gis.stackexchange.com/questions/66/trilateration-using-3-latitude-longitude-points-and-3-distances"
# using authalic sphere
# if using an ellipsoid this step is slightly different
# Convert geodetic Lat/Long to ECEF xyz
#   1. Convert Lat/Long to radians
#   2. Convert Lat/Long(radians) to ECEF
xA = earthR *(math.cos(math.radians(LatA)) * math.cos(math.radians(LonA)))
yA = earthR *(math.cos(math.radians(LatA)) * math.sin(math.radians(LonA)))
zA = earthR *(math.sin(math.radians(LatA)))

xB = earthR *(math.cos(math.radians(LatB)) * math.cos(math.radians(LonB)))
yB = earthR *(math.cos(math.radians(LatB)) * math.sin(math.radians(LonB)))
zB = earthR *(math.sin(math.radians(LatB)))

xC = earthR *(math.cos(math.radians(LatC)) * math.cos(math.radians(LonC)))
yC = earthR *(math.cos(math.radians(LatC)) * math.sin(math.radians(LonC)))
zC = earthR *(math.sin(math.radians(LatC)))

P1 = np.array([xA, yA, zA])
P2 = np.array([xB, yB, zB])
P3 = np.array([xC, yC, zC])

#from wikipedia
#transform to get circle 1 at origin
#transform to get circle 2 on x axis
ex = (P2 - P1)/(np.linalg.norm(P2 - P1))
i = np.dot(ex, P3 - P1)
ey = (P3 - P1 - i*ex)/(np.linalg.norm(P3 - P1 - i*ex))
ez = np.cross(ex,ey)
d = np.linalg.norm(P2 - P1)
j = np.dot(ey, P3 - P1)

#from wikipedia
#plug and chug using above values
x = (pow(DistA,2) - pow(DistB,2) + pow(d,2))/(2*d)
y = ((pow(DistA,2) - pow(DistC,2) + pow(i,2) + pow(j,2))/(2*j)) - ((i/j)*x)

# only one case shown here
z = np.sqrt(pow(DistA,2) - pow(x,2) - pow(y,2))

#triPt is an array with ECEF x,y,z of trilateration point
triPt = P1 + x*ex + y*ey + z*ez

#convert back to lat/long from ECEF
#convert to degrees
lat = math.degrees(math.asin(triPt[2] / earthR))
lon = math.degrees(math.atan2(triPt[1],triPt[0]))

print(lat, lon)

48.86691249963017 2.3553379517040205
